# Portfolio Trader

In [29]:
import ccxt
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os

In [30]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [31]:
load_dotenv()

True

In [32]:
api_key = os.getenv('BINANCE_TESTNET_API_KEY')
secret_key = os.getenv('BINANCE_TESTNET_API_SECRET')
portfolio_path = os.getenv('PORTFOLIO_PATH')

In [33]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

In [34]:
exchange = ccxt.binance()
exchange

ccxt.binance()

In [35]:
exchange.set_sandbox_mode(True)
exchange.apiKey = api_key
exchange.secret = secret_key

In [36]:
pt = PortfolioTrader(exchange=exchange)
pt

PortfolioTrader(exchange=Binance)

In [37]:
portfolio_path

'C:\\Users\\xjayc\\PycharmProjects\\pythonDataCookbook\\data\\Portfolio.csv'

In [38]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)

In [39]:
pt.target_weights

,t_weights
asset,
USDT,0.20000
BTC,0.10000
ETH,0.20000
BNB,0.20000
LTC,0.30000


In [40]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

    def load_raw_balances(self):
        self.raw_balances = self.exchange.fetch_balance()['info']['balances']
        df = pd.DataFrame(self.raw_balances)
        df = df[['asset','free']].copy()
        df.free = pd.to_numeric(df.free)
        df.set_index('asset', inplace=True)
        df = pd.merge(left=df, right=self.target_weights, how='outer', on='asset')
        df.fillna(0, inplace=True)
        df['ticker'] = np.nan
        df['usdt_price'] = np.nan
        df['usdt'] = np.nan
        df.loc[(df.free != 0) | (df.t_weights !=0)]
        self.rebalance_table = df.copy()

In [41]:
pt = PortfolioTrader(exchange=exchange)

In [42]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)
pt.target_weights

,t_weights
asset,
USDT,0.20000
BTC,0.10000
ETH,0.20000
BNB,0.20000
LTC,0.30000


In [43]:
pt.exchange.fetch_balance()['info']['balances']

[{'asset': 'BNB', 'free': '1000.00000000', 'locked': '0.00000000'},
 {'asset': 'BTC', 'free': '1.00017500', 'locked': '0.00000000'},
 {'asset': 'BUSD', 'free': '10000.00000000', 'locked': '0.00000000'},
 {'asset': 'ETH', 'free': '100.00000000', 'locked': '0.00000000'},
 {'asset': 'LTC', 'free': '500.00000000', 'locked': '0.00000000'},
 {'asset': 'TRX', 'free': '500000.00000000', 'locked': '0.00000000'},
 {'asset': 'USDT', 'free': '10000.41365706', 'locked': '0.00000000'},
 {'asset': 'XRP', 'free': '50000.00000000', 'locked': '0.00000000'}]

In [44]:
pt.load_raw_balances()

In [45]:
pt.rebalance_table

,free,t_weights,ticker,usdt_price,usdt
asset,,,,,
BNB,1000.00000,0.20000,NaN,NaN,NaN
BTC,1.00018,0.10000,NaN,NaN,NaN
BUSD,10000.00000,0.00000,NaN,NaN,NaN
ETH,100.00000,0.20000,NaN,NaN,NaN
LTC,500.00000,0.30000,NaN,NaN,NaN
TRX,500000.00000,0.00000,NaN,NaN,NaN
USDT,10000.41366,0.20000,NaN,NaN,NaN
XRP,50000.00000,0.00000,NaN,NaN,NaN


In [46]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

    def load_raw_balances(self):
        self.raw_balances = self.exchange.fetch_balance()['info']['balances']
        df = pd.DataFrame(self.raw_balances)
        df = df[['asset','free']].copy()
        df.free = pd.to_numeric(df.free)
        df.set_index('asset', inplace=True)
        df = pd.merge(left=df, right=self.target_weights, how='outer', on='asset')
        df.fillna(0, inplace=True)
        df['ticker'] = np.nan
        df['usdt_price'] = np.nan
        df['usdt'] = np.nan
        df.loc[(df.free != 0) | (df.t_weights !=0)]
        self.rebalance_table = df.copy()

    def convert_raw_balances(self):
        for symbol in self.rebalance_table.index:
            balance = self.rebalance_table.loc[symbol, 'free']
            if symbol == 'USDT':
                self.rebalance_table.loc[symbol, 'usdt_price'] = 1
            else:
                try:
                    target = 'USDT'
                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                except Exception as e:
                    try:
                        target = 'BTC'
                        self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                        self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                    except:
                        try:
                            target = 'ETH'
                            self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                            self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                        except:
                            try:
                                target = 'BNB'
                                self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                            except:
                                try:
                                    target = 'BUSD'
                                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                    self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                                except:
                                    print('{} can not be converted!'.format(symbol))
        self.rebalance_table['usdt'] = self.rebalance_table.free.mul(self.rebalance_table.usdt_price)
        self.balance = self.rebalance_table['usdt'].sum()

    def _fetch_and_convert(self, symbol, target, orig_symbol, step):
        try:
            ticker = symbol + '/' + target # symbol is base currency
            price = float(self.exchange.fetchTicker(ticker)['last'])
        except:
            ticker = target + '/' + symbol # symbol is quote currency
            price = 1/float(self.exchange.fetchTicker(ticker)['last'])

        if step == 'first':
            self.rebalance_table.loc[symbol, 'ticker'] = ticker
            self.rebalance_table.loc[symbol, 'ticker_price'] = price
            self.rebalance_table.loc[symbol, 'usdt_price'] = price
        elif step == 'second':
            self.rebalance_table.loc[orig_symbol, 'usdt_price'] *= price

In [47]:
pt = PortfolioTrader(exchange=exchange)
pt

PortfolioTrader(exchange=Binance)

In [48]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)

In [49]:
pt.load_raw_balances()

In [50]:
pt.convert_raw_balances()

BUSD can not be converted!


In [51]:
pt.rebalance_table

,free,t_weights,ticker,usdt_price,usdt,ticker_price
asset,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020


In [57]:
df = pt.rebalance_table.copy()
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price
asset,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020


In [58]:
df['init_weights'] = df.usdt.div(pt.balance)
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price,init_weights
asset,,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000,0.51435
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000,0.03862
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318,0.00005
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000,0.26748
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000,0.07790
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322,0.05170
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN,0.01636
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020,0.03354


In [59]:
df['target_usdt'] = df.t_weights.mul(pt.balance)
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price,init_weights,target_usdt
asset,,,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000,0.51435,122289.41644
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000,0.03862,61144.70822
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318,0.00005,0.00000
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000,0.26748,122289.41644
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000,0.07790,183434.12466
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322,0.05170,0.00000
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN,0.01636,122289.41644
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020,0.03354,0.00000


In [60]:
df['rebalance_usdt'] = df.target_usdt.sub(df.usdt)
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price,init_weights,target_usdt,rebalance_usdt
asset,,,,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000,0.51435,122289.41644,-192210.58356
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000,0.03862,61144.70822,37527.81599
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318,0.00005,0.00000,-31.77629
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000,0.26748,122289.41644,-41258.58356
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000,0.07790,183434.12466,135804.12466
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322,0.05170,0.00000,-31610.00000
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN,0.01636,122289.41644,112289.00278
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020,0.03354,0.00000,-20510.00000


In [61]:
df.sort_values(by='rebalance_usdt', ascending=True, inplace=True)
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price,init_weights,target_usdt,rebalance_usdt
asset,,,,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000,0.51435,122289.41644,-192210.58356
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000,0.26748,122289.41644,-41258.58356
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322,0.05170,0.00000,-31610.00000
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020,0.03354,0.00000,-20510.00000
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318,0.00005,0.00000,-31.77629
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000,0.03862,61144.70822,37527.81599
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN,0.01636,122289.41644,112289.00278
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000,0.07790,183434.12466,135804.12466


In [62]:
df['rebalance_units'] = df.rebalance_usdt.div(df.usdt_price)
df['traded_units'] = 0
df

,free,t_weights,ticker,usdt_price,usdt,ticker_price,init_weights,target_usdt,rebalance_usdt,rebalance_units,traded_units
asset,,,,,,,,,,,
BNB,1000.00000,0.20000,BNB/USDT,314.50000,314500.00000,314.50000,0.51435,122289.41644,-192210.58356,-611.16243,0
ETH,100.00000,0.20000,ETH/USDT,1635.48000,163548.00000,1635.48000,0.26748,122289.41644,-41258.58356,-25.22720,0
TRX,500000.00000,0.00000,TRX/USDT,0.06322,31610.00000,0.06322,0.05170,0.00000,-31610.00000,-500000.00000,0
XRP,50000.00000,0.00000,XRP/USDT,0.41020,20510.00000,0.41020,0.03354,0.00000,-20510.00000,-50000.00000,0
BUSD,10000.00000,0.00000,BNB/BUSD,0.00318,31.77629,0.00318,0.00005,0.00000,-31.77629,-10000.00000,0
BTC,1.00018,0.10000,BTC/USDT,23612.76000,23616.89223,23612.76000,0.03862,61144.70822,37527.81599,1.58930,0
USDT,10000.41366,0.20000,NaN,1.00000,10000.41366,NaN,0.01636,122289.41644,112289.00278,112289.00278,0
LTC,500.00000,0.30000,LTC/USDT,95.26000,47630.00000,95.26000,0.07790,183434.12466,135804.12466,1425.61542,0


In [65]:
df = df[['free','init_weights','t_weights','usdt_price','usdt','target_usdt','rebalance_usdt','rebalance_units','traded_units','ticker','ticker_price']].copy()
df

,free,init_weights,t_weights,usdt_price,usdt,target_usdt,rebalance_usdt,rebalance_units,traded_units,ticker,ticker_price
asset,,,,,,,,,,,
BNB,1000.00000,0.51435,0.20000,314.50000,314500.00000,122289.41644,-192210.58356,-611.16243,0,BNB/USDT,314.50000
ETH,100.00000,0.26748,0.20000,1635.48000,163548.00000,122289.41644,-41258.58356,-25.22720,0,ETH/USDT,1635.48000
TRX,500000.00000,0.05170,0.00000,0.06322,31610.00000,0.00000,-31610.00000,-500000.00000,0,TRX/USDT,0.06322
XRP,50000.00000,0.03354,0.00000,0.41020,20510.00000,0.00000,-20510.00000,-50000.00000,0,XRP/USDT,0.41020
BUSD,10000.00000,0.00005,0.00000,0.00318,31.77629,0.00000,-31.77629,-10000.00000,0,BNB/BUSD,0.00318
BTC,1.00018,0.03862,0.10000,23612.76000,23616.89223,61144.70822,37527.81599,1.58930,0,BTC/USDT,23612.76000
USDT,10000.41366,0.01636,0.20000,1.00000,10000.41366,122289.41644,112289.00278,112289.00278,0,NaN,NaN
LTC,500.00000,0.07790,0.30000,95.26000,47630.00000,183434.12466,135804.12466,1425.61542,0,LTC/USDT,95.26000


In [67]:
class PortfolioTrader():
    def __init__(self, exchange):
        self.exchange = exchange

    def __repr__(self):
        return 'PortfolioTrader(exchange={})'.format(self.exchange)

    def load_target_weights(self, filepath, weights_tolerance):
        self.filepath = filepath
        self.target_weights = pd.read_csv(filepath, usecols=[0,1])
        self.target_weights.columns = ['asset', 't_weights']
        self.target_weights.set_index('asset', inplace=True)
        if abs(self.target_weights.t_weights.sum() - 1 > weights_tolerance):
            print('Weights don\'t sum up to 1.')

    def load_raw_balances(self):
        self.raw_balances = self.exchange.fetch_balance()['info']['balances']
        df = pd.DataFrame(self.raw_balances)
        df = df[['asset','free']].copy()
        df.free = pd.to_numeric(df.free)
        df.set_index('asset', inplace=True)
        df = pd.merge(left=df, right=self.target_weights, how='outer', on='asset')
        df.fillna(0, inplace=True)
        df['ticker'] = np.nan
        df['usdt_price'] = np.nan
        df['usdt'] = np.nan
        df.loc[(df.free != 0) | (df.t_weights !=0)]
        self.rebalance_table = df.copy()

    def convert_raw_balances(self):
        for symbol in self.rebalance_table.index:
            balance = self.rebalance_table.loc[symbol, 'free']
            if symbol == 'USDT':
                self.rebalance_table.loc[symbol, 'usdt_price'] = 1
            else:
                try:
                    target = 'USDT'
                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                except Exception as e:
                    try:
                        target = 'BTC'
                        self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                        self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                    except:
                        try:
                            target = 'ETH'
                            self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                            self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                        except:
                            try:
                                target = 'BNB'
                                self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                            except:
                                try:
                                    target = 'BUSD'
                                    self._fetch_and_convert(symbol=symbol, target=target, orig_symbol=symbol, step='first')
                                    self._fetch_and_convert(symbol=symbol, target='USDT', orig_symbol=symbol, step='second')
                                except:
                                    print('{} can not be converted!'.format(symbol))
        self.rebalance_table['usdt'] = self.rebalance_table.free.mul(self.rebalance_table.usdt_price)
        self.balance = self.rebalance_table['usdt'].sum()

    def _fetch_and_convert(self, symbol, target, orig_symbol, step):
        try:
            ticker = symbol + '/' + target # symbol is base currency
            price = float(self.exchange.fetchTicker(ticker)['last'])
        except:
            ticker = target + '/' + symbol # symbol is quote currency
            price = 1/float(self.exchange.fetchTicker(ticker)['last'])

        if step == 'first':
            self.rebalance_table.loc[symbol, 'ticker'] = ticker
            self.rebalance_table.loc[symbol, 'ticker_price'] = price
            self.rebalance_table.loc[symbol, 'usdt_price'] = price
        elif step == 'second':
            self.rebalance_table.loc[orig_symbol, 'usdt_price'] *= price

    def calc_rebalance(self):
        df = self.rebalance_table.copy()

        df['init_weights'] = df.usdt.div(self.balance)
        df['target_usdt'] = df.t_weights.mul(self.balance)
        df['rebalance_usdt'] = df.target_usdt.sub(df.usdt)
        df.sort_values(by='rebalance_usdt', ascending=True, inplace=True)
        df['rebalance_units'] = df.rebalance_usdt.div(df.usdt_price)
        df['traded_units'] = 0
        df = df[['free','init_weights','t_weights','usdt_price','usdt','target_usdt','rebalance_usdt','rebalance_units','traded_units','ticker','ticker_price']].copy()

        self.rebalance_table = df.copy()

In [68]:
pt = PortfolioTrader(exchange=exchange)
pt

PortfolioTrader(exchange=Binance)

In [69]:
pt.load_target_weights(filepath=portfolio_path, weights_tolerance=0.01)

In [71]:
pt.load_raw_balances()

In [72]:
pt.calc_rebalance()

AttributeError: 'PortfolioTrader' object has no attribute 'balance'

In [73]:
pt.convert_raw_balances()

BUSD can not be converted!


In [74]:
pt.calc_rebalance()

In [75]:
pt.rebalance_table

,free,init_weights,t_weights,usdt_price,usdt,target_usdt,rebalance_usdt,rebalance_units,traded_units,ticker,ticker_price
asset,,,,,,,,,,,
BNB,1000.00000,0.51464,0.20000,315.00000,315000.00000,122416.41339,-192583.58661,-611.37647,0,BNB/USDT,315.00000
ETH,100.00000,0.26735,0.20000,1636.39000,163639.00000,122416.41339,-41222.58661,-25.19117,0,ETH/USDT,1636.39000
TRX,500000.00000,0.05166,0.00000,0.06324,31620.00000,0.00000,-31620.00000,-500000.00000,0,TRX/USDT,0.06324
XRP,50000.00000,0.03352,0.00000,0.41030,20515.00000,0.00000,-20515.00000,-50000.00000,0,XRP/USDT,0.41030
BUSD,10000.00000,0.00005,0.00000,0.00317,31.73596,0.00000,-31.73596,-10000.00000,0,BNB/BUSD,0.00317
BTC,1.00018,0.03863,0.10000,23641.78000,23645.91731,61208.20669,37562.28938,1.58881,0,BTC/USDT,23641.78000
USDT,10000.41366,0.01634,0.20000,1.00000,10000.41366,122416.41339,112415.99973,112415.99973,0,NaN,NaN
LTC,500.00000,0.07782,0.30000,95.26000,47630.00000,183624.62008,135994.62008,1427.61516,0,LTC/USDT,95.26000
